# 1 - PROBLEM

## Requisitos

[OK] 1. **Leitura de Dados:** Desenvolva um componente capaz de ler uma lista de objetos contidos em um arquivo JSON, representando informações sobre tema escolhidoes.

[Falta redução e filtro] 2. **Mapeamento, Filtro e Redução:** Implemente funcionalidades para realizar mapeamento, filtragem e redução dos dados, proporcionando uma análise mais refinada das informações contidas no conjunto de dados.

[OK] 3. **Manipulação de Dados Individuais:** Permita a leitura individual, atualização e exclusão de do arquivos, mantendo o arquivo JSON sempre atualizado.

[OK, dentro do possível] 4. **Validações de Operações:** Integre validações utilizando blocos try-except e raise para garantir a robustez das operações, prevenindo erros e assegurando a consistência dos dados.

[OK] 5. **Obtenção de Estatísticas Simples:** Desenvolva uma função para extrair dados estatísticos simples, como média, máximo e mínimo, por exemplo, sum item X nos dados de exatas.

[Falta] 6. **Identificação de Máximos/Mínimos com Detalhes:** Crie uma função que retorne uma lista de tuplas, contendXdo professor e o valor máximo (ou mínimo) de algum atributo numérico. Esta função deve ser configurável para fornecer estatísticas de máximo ou mínimo.

[OK] 7. **Exportação de Dados Estatísticos para CSV:** Implemente a capacidade de salvar os dados estatísticos obtidos em um arquivo CSV, permitindo uma análise posterior ou compartilhamento fácil dos resultados.

# 2 - IMPORTS

## 2.1 - Bibliotecas

In [7]:
import json
from funcoes import obter_opcoes
import csv
from IPython.display import clear_output

## 2.2 - Funções

### 2.2.1 - Leitura do Arquivo

In [8]:
def carrega_dados(path:str = 'data/receitas.json') -> list[dict]:
    try:
        with open(path, 'r', encoding='utf-8') as arquivo:
            dados = arquivo.read()
            return json.loads(dados)
    except FileNotFoundError:
        return []

### 2.2.2 - Salvamento do Arquivo

In [4]:
def salvar_dados(dados:list[dict], path:str = 'data/receitas.json') -> bool:
    try:
        with open(path, 'w', encoding='utf-8') as arquivo:
            arquivo.write(json.dumps(dados))
            return True
    except Exception:
        return False

### 2.2.3 - Visualização do Arquivo

In [5]:
def formata_dicionario(dados: dict) -> str:
    return ' | '.join([f'{k}: {v}' for k, v in dados.items()])

In [6]:
def formata_ingredientes(ingredientes: list) -> str:
    return '\n  '.join(map(formata_dicionario, ingredientes))

In [7]:
def formata_receita(receita):
    nome = receita['nome']
    ingredientes = formata_ingredientes(receita['ingredientes'])
    instrucoes = receita['instrucoes']
    return f'{nome}:\n Ingredientes:\n  {ingredientes}\n Instruções: {instrucoes}\n'

In [8]:
def formata_lista_receitas(lista_receitas):
    return '\n'.join(map(formata_receita, lista_receitas))

### 2.2.4 - Leitura do Arquivo * (está como leitura novamente, melhor outro título)

In [9]:
def retorna_dicionario(dado: dict, escolha=''):
    return dado if escolha in dado['nome'] else None

In [10]:
def verifica_escolha(data, escolha=''):
    resultado = []
    for dado in data:
        if retorna_dicionario(dado, escolha) is not None:
            resultado.append(dado)
    return resultado

### 2.2.5 - Adiciona Elemento

In [11]:
def obter_nome():
    while True:
        nome = input("Digite o nome da receita: ").strip().title()
        if nome:
            return nome
        else:
            print("Nome inválido. Por favor, digite novamente.")

In [12]:
def obter_ingrediente():
    while True:
        ingrediente = input("Digite o nome do ingrediente (ou 'fim' para encerrar): ").strip().title()
        if ingrediente.lower() == 'fim' or ingrediente:
            return ingrediente
        else:
            print("Ingrediente inválido. Por favor, digite novamente.")

In [13]:
def obter_quantidade():
    while True:
        quantidade = input("Digite a quantidade do ingrediente e sua unidade: ")
        return quantidade

In [14]:
def obter_custo():
    while True:
        custo = input("Digite o valor do ingrediente: ")
        if custo and validar_numero(custo):
            return float(custo)
        else:
            print("Custo inválido. Por favor, digite novamente.")

In [15]:
def validar_numero(valor):
    try:
        return float(valor) >= 0
    except ValueError:
        return False

In [16]:
def obter_instrucao():
    return input("Digite a instrução de preparo da receita: ").capitalize()

In [17]:
def criar_nova_receita(dados: list[dict]) -> bool:
    nova_receita = {'nome': obter_nome(), 'ingredientes': []}

    while True:
        ingrediente = obter_ingrediente()
        if ingrediente.lower() == 'fim':
            break

        quantidade = obter_quantidade()
        custo = obter_custo()

        nova_receita['ingredientes'].append({
            'nome': ingrediente,
            'quantidade': quantidade,
            'custo': custo
        })

    nova_receita['instrucoes'] = obter_instrucao()

    dados.append(nova_receita)
    return True

### 2.2.6 - Alterar Elementos


In [18]:
def busca(data: list[dict], input_name: str, field = 'nome') -> list[dict]:
  return [item for item in data if item[field].strip().title() == input_name.strip().title()]

In [19]:
def executa_alteracao(opc, opc_funcao, dado: dict) -> None:
    while True:
        opcao = obter_opcoes(opc, 'Escolha o campo a ser alterado')

        if opcao == 'F':
            break
        opc_funcao[opcao][1](dado)


In [20]:
def atualizar_nome_ingrediente(ingredientes):
    ingredientes['ingrediente'] = obter_ingrediente()

def atualizar_quantidade(ingredientes):
    ingredientes['quantidade'] = obter_quantidade()

def atualizar_custo(ingredientes):
    ingredientes['custo'] = obter_custo()


In [21]:
def atualizar_nome(receita: dict):
    receita['nome'] = obter_nome()

def atualizar_instrucao(receita: dict):
    receita['instrucoes'] = obter_instrucao()

def atualizar_ingrediente(receita: dict) -> bool:
    opc = {
        'I': 'Ingrediente',
        'Q': 'Quantidade',
        'C': 'Custo',
        'F': 'Finalizar'
    }

    opc_funcao = {
        'I' : ('ingrediente', atualizar_nome_ingrediente),
        'Q': ('quantidade', atualizar_quantidade),
        'C': ('custo', atualizar_custo)
    }
    ingredientes = receita['ingredientes']
    alterado = busca(ingredientes, obter_ingrediente(), 'ingrediente')

    if len(alterado) == 0:
        print('Não foi encontrado!')
        return False

    msg = f'Tem certeza que deseja alterar [{formata_ingredientes(alterado)}]'

    alterado = alterado[0]

    if obter_opcoes({'S': 'Sim', 'N': 'Não'}, msg) == 'S':
        executa_alteracao(opc, opc_funcao, alterado)
        return True
    else:
        return False

In [22]:
def alterar_receita(receitas: list[dict]) -> bool:
    alterado = busca(receitas, obter_nome(), 'nome')

    if len(alterado) == 0:
        print('Não foi encontrado!')
        return False

    alterado = alterado[0]

    opc = {
        'N': 'Nome',
        'I': 'Ingredientes',
        'P': 'Instruções de Preparo',
        'F': 'Finalizar'
    }

    opc_funcao = {
        'N' : ('nome', atualizar_nome),
        'I': ('ingredientes', atualizar_ingrediente),
        'P': ('instrucoes', atualizar_instrucao),
    }

    msg = f'Tem certeza que deseja alterar [{formata_receita(alterado)}]'

    if obter_opcoes({'S': 'Sim', 'N': 'Não'}, msg) == 'S':
        executa_alteracao(opc, opc_funcao, alterado)
        return True
    else:
        return False

### 2.2.7 Deletar

In [23]:
def deletar_ingrediente(receitas: list[dict]) -> bool:
    receita = busca(receitas, obter_nome(), 'nome')
    ingredientes = receita[0]['ingredientes']
    apagado = busca(ingredientes, obter_ingrediente(), 'ingrediente')
    
    if len(apagado) == 0:
        print('Não foi encontrado!')
        return False
    
    msg = f'Tem certeza que deseja excluir [{formata_ingredientes(apagado)}]'
    
    apagado = apagado[0]

    if obter_opcoes({'S': 'Sim', 'N': 'Não'}, msg) == 'S':
        ingredientes.remove(apagado)
        return True
    else:
        return False

In [24]:
def deletar_receita(receitas: list[dict]) -> bool:
    apagado = busca(receitas, obter_nome(), 'nome')
    
    if len(apagado) == 0:
        print('Não foi encontrado!')
        return False
    
    apagado = apagado[0]
    
    msg = f'Tem certeza que deseja excluir [{formata_receita(apagado)}]'
    
    if obter_opcoes({'S': 'Sim', 'N': 'Não'}, msg) == 'S':
        receitas.remove(apagado)
        return True
    else:
        return False

### 2.2.8 Calculando Custos

In [21]:
def calcula_custo(receita):
    custos_por_receita = []
    for item in receita:       
        custo = "{:.2f}".format(sum(ingrediente["custo"] for ingrediente in item["ingredientes"]))
        nome = item["nome"]
        custos_por_receita.append([nome, custo])
    return custos_por_receita

In [3]:
def receita_mais_cara(custo):
    return max(custo, key = lambda x: x[1])

In [27]:
def receita_mais_barata(custo):
    return min(custo, key = lambda x: x[1])

### 2.2.7 Salvar Dados Estatístico em um CSV

In [28]:
def salvar_dados_estatistico(dados) -> bool:
    custo = calcula_custo(dados)
    receita_mais_barata_ = receita_mais_barata(custo)
    receita_mais_cara_ = receita_mais_barata(custo)
    
    tabela = [['Nome', 'Custo Total', 'Receita Mais Cara', 'Receita Mais Barata']]
    tabela.append([custo[0][0], custo[0][1], receita_mais_cara_[0], receita_mais_barata_[0]])
    for i in custo[1::]:
        tabela.append(i)

    # cria o arquivo CSV
    arquivo = open('Dados Estatisticos.csv', 'w', encoding='utf-8-sig')

    # definindo as regras do nosso CSV:
    # ele será escrito no arquivo apontado pela variável 'arquivo'
    # seus elementos serão delimitados (delimiter) pelo símbolo ';'
    # suas linhas serão encerradas (lineterminator) por uma quebra de linha
    escritor = csv.writer(arquivo, delimiter=';', lineterminator='\n')

    # escreve uma lista de listas em formato CSV:
    escritor.writerows(tabela)

    # fecha e salva o arquivo
    arquivo.close()

    return True

# 3 - Code Start

## 3.1 - Carregamento de Dados

In [13]:
dados = carrega_dados()

In [16]:
dados

[{'nome': 'Lasanha',
  'ingredientes': [{'ingrediente': 'Massa de Lasanha',
    'quantidade': '4un',
    'custo': 2.0},
   {'ingrediente': 'molho de tomate', 'quantidade': '500ml', 'custo': 5.0},
   {'ingrediente': 'queijo', 'quantidade': '200g', 'custo': 4.0},
   {'ingrediente': 'manteiga', 'quantidade': '50g', 'custo': 1.0}],
  'instrucoes': 'Cozinhe o macarrão, faça o molho de tomate, cozinhe a carne, monte as camadas e asse no forno.'},
 {'nome': 'Salada Caesar',
  'ingredientes': [{'ingrediente': 'alface romana',
    'quantidade': '1 cabeça',
    'custo': 2.0},
   {'ingrediente': 'croutons', 'quantidade': '1 xícara', 'custo': 1.5},
   {'ingrediente': 'queijo parmesão', 'quantidade': '50g', 'custo': 5.0},
   {'ingrediente': 'peito de frango grelhado',
    'quantidade': '300g',
    'custo': 6.0},
   {'ingrediente': 'molho Caesar', 'quantidade': '150ml', 'custo': 2.5}],
  'instrucoes': 'Misture os ingredientes, adicione o frango grelhado e regue com molho Caesar.'},
 {'nome': 'Sopa d

## 3.2 - Visualizar os dados carregados

In [31]:
resultado_formatado = formata_lista_receitas(dados)

In [32]:
print(resultado_formatado)

Lasanha:
 Ingredientes:
  ingrediente: Massa de Lasanha | quantidade: 4un | custo: 2.0
  ingrediente: molho de tomate | quantidade: 500ml | custo: 5.0
  ingrediente: queijo | quantidade: 200g | custo: 4.0
  ingrediente: manteiga | quantidade: 50g | custo: 1.0
 Instruções: Cozinhe o macarrão, faça o molho de tomate, cozinhe a carne, monte as camadas e asse no forno.

Salada Caesar:
 Ingredientes:
  ingrediente: alface romana | quantidade: 1 cabeça | custo: 2.0
  ingrediente: croutons | quantidade: 1 xícara | custo: 1.5
  ingrediente: queijo parmesão | quantidade: 50g | custo: 5.0
  ingrediente: peito de frango grelhado | quantidade: 300g | custo: 6.0
  ingrediente: molho Caesar | quantidade: 150ml | custo: 2.5
 Instruções: Misture os ingredientes, adicione o frango grelhado e regue com molho Caesar.

Sopa de Abóbora:
 Ingredientes:
  ingrediente: abóbora | quantidade: 1 kg | custo: 2.5
  ingrediente: cebola | quantidade: 1 | custo: 0.5
  ingrediente: alho | quantidade: 2 dentes | custo:

## 3.3 - Leitura Individual

In [33]:
x = verifica_escolha(dados, 'Salada Caesar')

In [34]:
x

[{'nome': 'Salada Caesar',
  'ingredientes': [{'ingrediente': 'alface romana',
    'quantidade': '1 cabeça',
    'custo': 2.0},
   {'ingrediente': 'croutons', 'quantidade': '1 xícara', 'custo': 1.5},
   {'ingrediente': 'queijo parmesão', 'quantidade': '50g', 'custo': 5.0},
   {'ingrediente': 'peito de frango grelhado',
    'quantidade': '300g',
    'custo': 6.0},
   {'ingrediente': 'molho Caesar', 'quantidade': '150ml', 'custo': 2.5}],
  'instrucoes': 'Misture os ingredientes, adicione o frango grelhado e regue com molho Caesar.'}]

## 3.4 - Adiciona Elemento

In [35]:
#criar_nova_receita(dados)

## 3.5 - Alterar Receita

In [36]:
#alterar_receita(dados)

False

In [37]:
print(formata_lista_receitas(dados))

Lasanha:
 Ingredientes:
  ingrediente: Massa de Lasanha | quantidade: 4un | custo: 2.0
  ingrediente: molho de tomate | quantidade: 500ml | custo: 5.0
  ingrediente: queijo | quantidade: 200g | custo: 4.0
  ingrediente: manteiga | quantidade: 50g | custo: 1.0
 Instruções: Cozinhe o macarrão, faça o molho de tomate, cozinhe a carne, monte as camadas e asse no forno.

Salada Caesar:
 Ingredientes:
  ingrediente: alface romana | quantidade: 1 cabeça | custo: 2.0
  ingrediente: croutons | quantidade: 1 xícara | custo: 1.5
  ingrediente: queijo parmesão | quantidade: 50g | custo: 5.0
  ingrediente: peito de frango grelhado | quantidade: 300g | custo: 6.0
  ingrediente: molho Caesar | quantidade: 150ml | custo: 2.5
 Instruções: Misture os ingredientes, adicione o frango grelhado e regue com molho Caesar.

Sopa de Abóbora:
 Ingredientes:
  ingrediente: abóbora | quantidade: 1 kg | custo: 2.5
  ingrediente: cebola | quantidade: 1 | custo: 0.5
  ingrediente: alho | quantidade: 2 dentes | custo:

## 3.6 - Deletar Elemento

In [38]:
#deletar_receita(dados)

In [39]:
#deletar_ingrediente(dados)

In [40]:
print(formata_lista_receitas(dados))

Lasanha:
 Ingredientes:
  ingrediente: Massa de Lasanha | quantidade: 4un | custo: 2.0
  ingrediente: molho de tomate | quantidade: 500ml | custo: 5.0
  ingrediente: queijo | quantidade: 200g | custo: 4.0
  ingrediente: manteiga | quantidade: 50g | custo: 1.0
 Instruções: Cozinhe o macarrão, faça o molho de tomate, cozinhe a carne, monte as camadas e asse no forno.

Salada Caesar:
 Ingredientes:
  ingrediente: alface romana | quantidade: 1 cabeça | custo: 2.0
  ingrediente: croutons | quantidade: 1 xícara | custo: 1.5
  ingrediente: queijo parmesão | quantidade: 50g | custo: 5.0
  ingrediente: peito de frango grelhado | quantidade: 300g | custo: 6.0
  ingrediente: molho Caesar | quantidade: 150ml | custo: 2.5
 Instruções: Misture os ingredientes, adicione o frango grelhado e regue com molho Caesar.

Sopa de Abóbora:
 Ingredientes:
  ingrediente: abóbora | quantidade: 1 kg | custo: 2.5
  ingrediente: cebola | quantidade: 1 | custo: 0.5
  ingrediente: alho | quantidade: 2 dentes | custo:

## 3.7 - Dados estatísticos

In [41]:
salvar_dados_estatistico(dados)

True

# 4 - Menu Principal

In [42]:
def sair(dados):
    pass

In [43]:
opc = {
    'C': 'Criar nova receita', 
    'A': 'Alterar receita', 
    'DR': 'Deletar receita', 
    'DI': 'Deletar ingrediente', 
    'B': 'Buscar', 
    'ET': 'Exibir Todos',
    'D': 'Obter Dados Estatísticos .csv',
    'S': 'Sair'
}

opc_func = {
    'C': criar_nova_receita,
    'A': alterar_receita,
    'DR': deletar_receita,
    'DI': deletar_ingrediente,
    'B': lambda dados: print(formata_lista_receitas(busca(dados, obter_nome()))),
    'ET': lambda dados: print(formata_lista_receitas(dados)),
    'D': salvar_dados_estatistico,
    'S': sair
}


In [46]:
saida = False
while not saida:
    #clear_output(wait=True)
    dados = carrega_dados()
    opcao = obter_opcoes(opc, 'Escolha uma ação')
    opc_func[opcao](dados)
    
    salvar_dados(dados)

    if opcao == 'S':
        saida = True


Lasanha2:
 Ingredientes:
  ingrediente: Massa de Lasanha | quantidade: 4un | custo: 2.0
  ingrediente: molho de tomate | quantidade: 500ml | custo: 5.0
  ingrediente: queijo | quantidade: 200g | custo: 4.0
  ingrediente: manteiga | quantidade: 50g | custo: 1.0
 Instruções: Cozinhe o macarrão, faça o molho de tomate, cozinhe a carne, monte as camadas e asse no forno.

